# 06 - Inventory Optimization

## Purpose
Optimize stock levels and reorder points for the Smart Inventory Manager.

## Sections
1. Current Inventory Status
2. Stock Velocity Calculation
3. Reorder Point Optimization
4. Reorder Quantity Optimization
5. Stockout Risk Analysis
6. Dead Stock Deep Dive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')

# Load data
DATA_DIR = Path('../..') / 'ml' / 'data' / 'processed'

products = pd.read_csv(DATA_DIR / 'products.csv')
inventory = pd.read_csv(DATA_DIR / 'inventory.csv')
orders = pd.read_csv(DATA_DIR / 'orders.csv')
order_items = pd.read_csv(DATA_DIR / 'order_items.csv')

orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])

# Merge for analysis
full_data = orders.merge(order_items, on='OrderID').merge(products, on='ProductID').merge(inventory, on='ProductID')
print(f"Full dataset: {len(full_data):,} rows")

## 1. Current Inventory Status

In [ ]:
# Overall inventory status
print("=" * 50)
print("INVENTORY STATUS SUMMARY")
print("=" * 50)

total_products = len(inventory)
in_stock = (inventory['Current_Stock'] > 0).sum()
out_of_stock = (inventory['Current_Stock'] == 0).sum()
low_stock = ((inventory['Current_Stock'] > 0) & (inventory['Current_Stock'] <= inventory['Reorder_Level'])).sum()

print(f"\nTotal Products: {total_products:,}")
print(f"In Stock: {in_stock:,} ({in_stock/total_products*100:.1f}%)")
print(f"Low Stock: {low_stock:,} ({low_stock/total_products*100:.1f}%)")
print(f"Out of Stock: {out_of_stock:,} ({out_of_stock/total_products*100:.1f}%)")

# Inventory value
inv_with_price = inventory.merge(products[['ProductID', 'Cost_Price']], on='ProductID')
inv_with_price['Stock_Value'] = inv_with_price['Current_Stock'] * inv_with_price['Cost_Price']
total_value = inv_with_price['Stock_Value'].sum()
print(f"\nTotal Inventory Value: ${total_value:,.2f}")

In [ ]:
# Stock status distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Stock status pie
status_counts = inventory['Stock_Status'].value_counts()
axes[0].pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Stock Status Distribution')

# Current stock distribution
axes[1].hist(inventory['Current_Stock'], bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(inventory['Current_Stock'].mean(), color='red', linestyle='--', label=f'Mean: {inventory["Current_Stock"].mean():.0f}')
axes[1].set_title('Current Stock Level Distribution')
axes[1].set_xlabel('Stock Level')
axes[1].legend()

plt.tight_layout()
plt.show()

## 2. Stock Velocity Calculation

In [ ]:
# Calculate sales velocity for each product
cutoff_30 = orders['OrderDate'].max() - timedelta(days=30)
cutoff_90 = orders['OrderDate'].max() - timedelta(days=90)

# 30-day velocity
recent_sales = full_data[full_data['OrderDate'] >= cutoff_30].copy()
velocity_30 = recent_sales.groupby('ProductID')['Quantity'].sum() / 30
velocity_30.name = 'Velocity_30d'

# 90-day velocity
sales_90 = full_data[full_data['OrderDate'] >= cutoff_90].copy()
velocity_90 = sales_90.groupby('ProductID')['Quantity'].sum() / 90
velocity_90.name = 'Velocity_90d'

# Combine with inventory
inv_analysis = inventory.merge(pd.DataFrame(velocity_30), left_on='ProductID', right_index=True, how='left')
inv_analysis = inv_analysis.merge(pd.DataFrame(velocity_90), left_on='ProductID', right_index=True, how='left')
inv_analysis = inv_analysis.merge(products[['ProductID', 'ProductName', 'Category', 'Cost_Price']], on='ProductID')

inv_analysis['Velocity_30d'] = inv_analysis['Velocity_30d'].fillna(0)
inv_analysis['Velocity_90d'] = inv_analysis['Velocity_90d'].fillna(0)

# Days of stock remaining
inv_analysis['Days_Of_Stock'] = np.where(
    inv_analysis['Velocity_30d'] > 0,
    inv_analysis['Current_Stock'] / inv_analysis['Velocity_30d'],
    np.inf
)

print("Stock Velocity Summary:")
display(inv_analysis[['ProductID', 'ProductName', 'Current_Stock', 'Velocity_30d', 'Velocity_90d', 'Days_Of_Stock']].head(10))

In [ ]:
# Velocity distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 30-day velocity histogram
axes[0].hist(inv_analysis['Velocity_30d'][inv_analysis['Velocity_30d'] > 0], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('30-Day Sales Velocity Distribution')
axes[0].set_xlabel('Units per Day')

# Velocity vs Stock scatter
axes[1].scatter(inv_analysis['Velocity_30d'], inv_analysis['Current_Stock'], alpha=0.5)
axes[1].set_title('Stock Level vs Sales Velocity')
axes[1].set_xlabel('Velocity (units/day)')
axes[1].set_ylabel('Current Stock')

plt.tight_layout()
plt.show()

## 3. Reorder Point Optimization

In [ ]:
# Calculate optimal reorder points
# Reorder Point = (Average Daily Demand * Lead Time) + Safety Stock

LEAD_TIME_DAYS = 7  # Assume 7 days lead time
SERVICE_LEVEL = 0.95  # 95% service level
Z_SCORE = 1.65  # Z-score for 95% service level

# Calculate demand variability
daily_demand = full_data.groupby(['ProductID', full_data['OrderDate'].dt.date])['Quantity'].sum().reset_index()
daily_demand.columns = ['ProductID', 'Date', 'Quantity']

demand_stats = daily_demand.groupby('ProductID')['Quantity'].agg(['mean', 'std']).reset_index()
demand_stats.columns = ['ProductID', 'Avg_Daily_Demand', 'Std_Daily_Demand']
demand_stats['Std_Daily_Demand'] = demand_stats['Std_Daily_Demand'].fillna(0)

inv_analysis = inv_analysis.merge(demand_stats, on='ProductID', how='left')
inv_analysis['Avg_Daily_Demand'] = inv_analysis['Avg_Daily_Demand'].fillna(0)
inv_analysis['Std_Daily_Demand'] = inv_analysis['Std_Daily_Demand'].fillna(0)

# Calculate optimal reorder point
inv_analysis['Safety_Stock'] = Z_SCORE * inv_analysis['Std_Daily_Demand'] * np.sqrt(LEAD_TIME_DAYS)
inv_analysis['Optimal_Reorder_Point'] = (
    inv_analysis['Avg_Daily_Demand'] * LEAD_TIME_DAYS + inv_analysis['Safety_Stock']
).round(0)

# Compare with current reorder level
inv_analysis['ROP_Difference'] = inv_analysis['Optimal_Reorder_Point'] - inv_analysis['Reorder_Level']

print("Reorder Point Analysis:")
display(inv_analysis[['ProductID', 'ProductName', 'Reorder_Level', 'Optimal_Reorder_Point', 'ROP_Difference']].head(10))

In [ ]:
# Products needing reorder point adjustment
needs_adjustment = inv_analysis[abs(inv_analysis['ROP_Difference']) > 5]

print(f"\nProducts needing ROP adjustment: {len(needs_adjustment):,}")
print(f"Products with ROP too low: {(needs_adjustment['ROP_Difference'] > 0).sum()}")
print(f"Products with ROP too high: {(needs_adjustment['ROP_Difference'] < 0).sum()}")

# Visualize
plt.figure(figsize=(10, 6))
plt.scatter(inv_analysis['Reorder_Level'], inv_analysis['Optimal_Reorder_Point'], alpha=0.5)
plt.plot([0, 100], [0, 100], 'r--', label='Perfect Match')
plt.xlabel('Current Reorder Level')
plt.ylabel('Optimal Reorder Point')
plt.title('Current vs Optimal Reorder Points')
plt.legend()
plt.show()

## 4. Economic Order Quantity (EOQ)

In [ ]:
# Calculate EOQ
# EOQ = sqrt((2 * Annual Demand * Ordering Cost) / Holding Cost per Unit)

ORDERING_COST = 50  # Cost per order
HOLDING_COST_PCT = 0.25  # 25% of product cost per year

inv_analysis['Annual_Demand'] = inv_analysis['Avg_Daily_Demand'] * 365
inv_analysis['Holding_Cost'] = inv_analysis['Cost_Price'] * HOLDING_COST_PCT

inv_analysis['EOQ'] = np.where(
    (inv_analysis['Annual_Demand'] > 0) & (inv_analysis['Holding_Cost'] > 0),
    np.sqrt((2 * inv_analysis['Annual_Demand'] * ORDERING_COST) / inv_analysis['Holding_Cost']),
    inv_analysis['Restock_Quantity']
).round(0)

print("Economic Order Quantity Analysis:")
display(inv_analysis[['ProductID', 'ProductName', 'Restock_Quantity', 'EOQ', 'Annual_Demand']].head(10))

## 5. Stockout Risk Analysis

In [ ]:
# Identify products at risk of stockout
inv_analysis['Stockout_Risk'] = np.where(
    inv_analysis['Days_Of_Stock'] <= LEAD_TIME_DAYS,
    'Critical',
    np.where(
        inv_analysis['Days_Of_Stock'] <= LEAD_TIME_DAYS * 2,
        'High',
        np.where(
            inv_analysis['Days_Of_Stock'] <= LEAD_TIME_DAYS * 4,
            'Medium',
            'Low'
        )
    )
)

risk_summary = inv_analysis['Stockout_Risk'].value_counts()
print("Stockout Risk Summary:")
print(risk_summary)

# Critical items
critical_items = inv_analysis[inv_analysis['Stockout_Risk'] == 'Critical'].sort_values('Days_Of_Stock')
print(f"\nCritical Items ({len(critical_items)}):")
display(critical_items[['ProductID', 'ProductName', 'Current_Stock', 'Velocity_30d', 'Days_Of_Stock']].head(10))

In [ ]:
# Visualize stockout risk
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Risk distribution
colors = {'Critical': 'red', 'High': 'orange', 'Medium': 'yellow', 'Low': 'green'}
risk_order = ['Critical', 'High', 'Medium', 'Low']
risk_counts = [risk_summary.get(r, 0) for r in risk_order]
axes[0].bar(risk_order, risk_counts, color=[colors[r] for r in risk_order])
axes[0].set_title('Stockout Risk Distribution')
axes[0].set_ylabel('Number of Products')

# Days of stock distribution
finite_days = inv_analysis[inv_analysis['Days_Of_Stock'] < 1000]['Days_Of_Stock']
axes[1].hist(finite_days, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(LEAD_TIME_DAYS, color='red', linestyle='--', label=f'Lead Time ({LEAD_TIME_DAYS} days)')
axes[1].set_title('Days of Stock Distribution')
axes[1].set_xlabel('Days of Stock')
axes[1].legend()

plt.tight_layout()
plt.show()

## 6. Dead Stock Analysis

In [ ]:
# Find last sale date for each product
last_sale = full_data.groupby('ProductID')['OrderDate'].max().reset_index()
last_sale.columns = ['ProductID', 'Last_Sale_Date']

inv_analysis = inv_analysis.merge(last_sale, on='ProductID', how='left')

# Calculate days since last sale
max_date = orders['OrderDate'].max()
inv_analysis['Days_Since_Sale'] = (max_date - inv_analysis['Last_Sale_Date']).dt.days

# Identify dead stock (no sales in 90+ days)
DEAD_STOCK_THRESHOLD = 90

dead_stock = inv_analysis[
    (inv_analysis['Days_Since_Sale'] >= DEAD_STOCK_THRESHOLD) | (inv_analysis['Last_Sale_Date'].isna())
].copy()
dead_stock = dead_stock[dead_stock['Current_Stock'] > 0]

dead_stock['Stock_Value'] = dead_stock['Current_Stock'] * dead_stock['Cost_Price']

print(f"Dead Stock Summary ({DEAD_STOCK_THRESHOLD}+ days without sales):")
print(f"  Products: {len(dead_stock):,}")
print(f"  Total Units: {dead_stock['Current_Stock'].sum():,}")
print(f"  Total Value: ${dead_stock['Stock_Value'].sum():,.2f}")

display(dead_stock[['ProductID', 'ProductName', 'Category', 'Current_Stock', 'Days_Since_Sale', 'Stock_Value']].sort_values('Stock_Value', ascending=False).head(10))

In [ ]:
# Dead stock by category
dead_by_category = dead_stock.groupby('Category').agg({
    'ProductID': 'count',
    'Current_Stock': 'sum',
    'Stock_Value': 'sum'
}).reset_index()
dead_by_category.columns = ['Category', 'Products', 'Units', 'Value']
dead_by_category = dead_by_category.sort_values('Value', ascending=False)

plt.figure(figsize=(12, 6))
plt.barh(dead_by_category['Category'], dead_by_category['Value'] / 1000)
plt.xlabel('Dead Stock Value ($K)')
plt.title('Dead Stock Value by Category')
plt.tight_layout()
plt.show()

In [ ]:
# Recommendations
print("\n" + "="*60)
print("INVENTORY OPTIMIZATION RECOMMENDATIONS")
print("="*60)

print(f"\n1. REORDER POINTS")
print(f"   - {len(needs_adjustment)} products need reorder point adjustment")
print(f"   - Average recommended increase: {needs_adjustment[needs_adjustment['ROP_Difference'] > 0]['ROP_Difference'].mean():.1f} units")

print(f"\n2. STOCKOUT RISK")
print(f"   - {len(critical_items)} products at CRITICAL risk")
print(f"   - Immediate reorder needed for these items")

print(f"\n3. DEAD STOCK")
print(f"   - ${dead_stock['Stock_Value'].sum():,.2f} tied up in dead inventory")
print(f"   - Consider clearance sales or bundling")

print(f"\n4. EOQ OPTIMIZATION")
eoq_savings = abs(inv_analysis['EOQ'] - inv_analysis['Restock_Quantity']).sum()
print(f"   - Implementing EOQ could optimize {eoq_savings:.0f} units in ordering")